In [1]:
!git clone https://github.com/san-deep-reddy/adapters.git

Cloning into 'adapters'...
remote: Enumerating objects: 2720, done.
remote: Counting objects: 100% (1904/1904), done.
remote: Compressing objects: 100% (815/815), done.
remote: Total 2720 (delta 1344), reused 1253 (delta 1034), pack-reused 816
Receiving objects: 100% (2720/2720), 9.78 MiB | 33.48 MiB/s, done.
Resolving deltas: 100% (1678/1678), done.


In [2]:
import sys, torch
sys.path.append("/kaggle/working/adapters/src/")
import adapters
import numpy as np
import pandas as pd
import torch.nn as nn
import adapters.composition as ac
from adapters.composition import Fuse
from adapters.heads import PredictionHead
from adapters import AutoAdapterModel, AdapterTrainer, SeqBnConfig
from transformers import AutoTokenizer, TrainingArguments, EvalPrediction, default_data_collator, Trainer
from datasets import Dataset, load_dataset

2024-05-27 04:29:51.783254: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-27 04:29:51.783356: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-27 04:29:51.936084: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
model_path = "mse30/bart-base-finetuned-pubmed"
model = AutoAdapterModel.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

config.json:   0%|          | 0.00/1.71k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/558M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


tokenizer_config.json:   0%|          | 0.00/331 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [4]:
import copy
model_copy = copy.deepcopy(model)

In [5]:
seq_config = SeqBnConfig(reduction_factor=16, use_gating=True)
model.add_adapter("adapter2", config=seq_config)
model.delete_head('default')
model.add_multiple_choice_head("adapter2", layers=2, num_choices=1)
model.train_adapter(['adapter2'])

In [6]:
# import json
# import pandas as pd
# import numpy as np

# json_file_path = "/kaggle/input/negative-samples/negative_samples_count64.json"
# with open(json_file_path, 'r') as file:
#     negative_samples = json.load(file)

# num_negative_samples = 64
# batch_size = (1 + num_negative_samples) * 4
# num_positive_samples_per_batch = batch_size // (1 + num_negative_samples)

# df_list = []

# # Define the relationships
# relationships = ["interacts with", "is related to", "interacts", "has function", "is a", "treats"]

# # Generate samples from negative_samples
# for relation in relationships:
#     for entity1, values in negative_samples.get(relation, {}).items():
#         for entity2, negative_entities in values.items():
#             samples = [f"{entity1} {relation} {entity2}"]
#             samples.extend([f"{entity1} {relation} {neg_entity}" for neg_entity in negative_entities[:num_negative_samples]])
#             df_list.append(samples)

# # Shuffle the samples before creating the DataFrame
# random.shuffle(df_list)

# # Flatten the list of lists
# flattened_list = [item for sublist in df_list for item in sublist]

# # Create a DataFrame from the flattened list
# df = pd.DataFrame(flattened_list, columns=['text'])

# # Generate labels pattern
# pattern = [1] + [0]*num_negative_samples
# labels = (pattern * (len(df) // len(pattern) + 1))[:len(df)]

# # Add labels to the DataFrame
# df['labels'] = labels

# def generate_batch_indices(start_index, num_positive_samples, num_negative_samples, batch_size, max_index):
#     indices = []
#     for i in range(num_positive_samples):
#         idx = start_index + i * (num_negative_samples + 1)
#         if idx < max_index:
#             indices.append(idx)
#     for j in range(1, num_negative_samples + 1):
#         for i in range(num_positive_samples):
#             idx = start_index + i * (num_negative_samples + 1) + j
#             if idx < max_index:
#                 indices.append(idx)
#     return indices

# # Generate indices for all batches
# all_indices = []
# max_index = len(df)
# for start_index in range(0, max_index, batch_size):
#     batch_indices = generate_batch_indices(start_index, num_positive_samples_per_batch, num_negative_samples, batch_size, max_index)
    
#     # Check if the batch has the required number of positive samples
#     if len(batch_indices) == batch_size:
#         all_indices.extend(batch_indices)

# # Reorder the DataFrame based on the generated indices
# df = df.iloc[all_indices].reset_index(drop=True)

In [7]:
df = pd.read_csv("/kaggle/input/negative-samples/64negsamples.csv")
num_negative_samples = 64
batch_size = (1 + num_negative_samples) * 4
num_positive_samples_per_batch = batch_size // (1 + num_negative_samples)

In [9]:
def get_train_test_sizes(df_length, batch_size, num_negative_samples):
    train_split = 0.8  # Desired train split
    test_split = 0.2   # Desired test split

    # Calculate train and test sizes
    train_size = int(np.floor(train_split * df_length / (batch_size * (1 + num_negative_samples))) * batch_size * (1 + num_negative_samples))
    test_size = int(np.floor(test_split * df_length / (batch_size * (1 + num_negative_samples))) * batch_size * (1 + num_negative_samples))

    return train_size, test_size

# Usage
train_size, test_size = get_train_test_sizes(len(df), batch_size, num_negative_samples)
dataset = Dataset.from_pandas(df).train_test_split(train_size=train_size, test_size=test_size, shuffle=False)

In [ ]:
def encode_batch(batch):
    """Encodes a batch of input data using the model tokenizer."""
    return tokenizer(batch["text"], padding=True, return_tensors="pt")

dataset = dataset.map(encode_batch, batched=True)

Map:   0%|          | 0/25012000 [00:00<?, ? examples/s]

In [ ]:
max_token_length_train = max([len(batch["input_ids"]) for batch in dataset["train"]])
print(max_token_length_train)
max_token_length_eval = max([len(batch["input_ids"]) for batch in dataset["test"]])
print(max_token_length_eval)
max_token_length = max(max_token_length_train, max_token_length_eval)
print(max_token_length)

def pad_to_max_length(example):
    input_ids = torch.tensor(example["input_ids"])
    attention_mask = torch.tensor(example["attention_mask"])
    padded_input_ids = torch.nn.functional.pad(input_ids, (0, max_token_length - input_ids.shape[0]), value=0)
    padded_attention_mask = torch.nn.functional.pad(attention_mask, (0, max_token_length - attention_mask.shape[0]), value=0)
    return {"input_ids": padded_input_ids, "attention_mask": padded_attention_mask, "labels": example["labels"]}

dataset["train"] = dataset["train"].map(pad_to_max_length)
dataset["test"] = dataset["test"].map(pad_to_max_length)

In [ ]:
from transformers import EarlyStoppingCallback

early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=3,  # Stop training after 3 epochs without improvement
    early_stopping_threshold=0.001  # Function to calculate the monitored quantity (e.g., evaluation loss)
)


training_args = TrainingArguments(
    output_dir="./",
    num_train_epochs=100,
    per_device_train_batch_size=520,
    per_device_eval_batch_size=520,
    learning_rate=2e-5,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    report_to="tensorboard",
    logging_strategy="epoch",
load_best_model_at_end = True,
do_train true
do_ev)

def compute_accuracy(p: EvalPrediction):
    print(p, "\n", type())
    print(p.predictions, "\n", p.redictions.shape)
    print(p.label_ids, "\n")
    preds = np.argmax(p.predictions[0], axis=1).flatten()
    return {"acc": (preds == p.label_ids).mean()}

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
#     compute_metrics=compute_accuracy,
    #callbacks=[early_stopping_callback]
)

trainer.train()